In [1]:
import sys
sys.path.append('../')

In [2]:
from tank import Tank, Hole, create_holes
import ipywidgets as widgets
from ipycanvas import canvas
from IPython.display import display, Markdown, Latex
import time
from demo import *
from pythreejs import *

In [ ]:
c = canvas.Canvas(width=350, height=250)
m = Tank(create_holes(25, 2), 0.06, c=c)

scene = Scene(children=[m.tank_pivot, m.water_pivot, camera, AmbientLight(color='#FFFFFF')], background=None)
m.draw_holes3D(scene)

renderer = Renderer(camera=camera,
                    scene=scene,
                    alpha=True,
                    controls=[OrbitControls(controlling=camera)],
                    clearOpacity=1,
                    clearColor='green',
                    width=600, height=200)

demo = Demo(m.params, m, drawable=widgets.HBox([c, renderer]))

In [ ]:
demo.show()